# 3章 関数の構文

## パターンマッチ

関数の引数に具体的な値を入れることでif/else書かなくてよくなる  
具体的な値ではなく小文字から始まる名前にすると任意の値に合致するようになる

In [2]:
sayMe :: Int -> String

sayMe 1 = "one"
sayMe 2 = "two"
sayMe 3 = "three"
sayMe x = "uncool"

sayMe 2
sayMe 18

"two"

"uncool"

関数を再起的に定義することで階乗の計算もできる

In [4]:
factorial :: Int -> Int
factorial 0 = 1
factorial n = n * factorial (n-1)

factorial 3

6

### タプルのパターンマッチ

パターンマッチを知らないハダ

In [5]:
addVectors :: (Double, Double) -> (Double, Double) -> (Double, Double)
addVectors a b = (fst a + fst b, snd a + snd b)

addVectors (3.8, 4.2) (1.2, 0.5)

(5.0,4.7)

パターンマッチを知ったハダ  
x1, x2 , y1, y2はそれぞれ任意の値

In [6]:
addVectors :: (Double, Double) -> (Double, Double) -> (Double, Double)
addVectors (x1, y1) (x2, y2) = (x1 + x2, y1 + y2)

addVectors (3.8, 4.2) (1.2, 0.5)

(5.0,4.7)

トリプルのタプルに対して要素を取り出す

In [7]:
first :: (a, b, c) -> a
first (x, _, _) = x

second :: (a, b, c) -> b
second (_, y, _) = y

third :: (a, b, c) -> c
third (_, _, z) = z

first (1,2,3)
second (1,2,3)
third (1,2,3)

1

2

3

### リストのパターンマッチ

In [10]:
let xs = [(1,3),(4,3),(2,4),(5,3),(5,6),(3,1)]
[a+b | (a, b) <- xs]

[4,7,6,8,11,4]

リスト内包表記のパターンマッチは失敗したら次の要素に進むだけ

In [9]:
let xs = [(1,3),(4,3),(2,4),(5,3),(5,6),(3,1)]
[x*100+3 | (x, 3) <- xs]

[103,403,503]

In [11]:
head' :: [a] -> a
head' [] = error "not empty list"

-- [1,2,3]は1:2:3:[]の構文糖衣
head' (x:_) = x

head' [4,5,6]
head' []

4

: 

パターンマッチでは++演算子(二つのリストをつなげる)は使えない

### asパターン

パターンマッチの対象になった元の値を参照したい時に使う  
`all@(x:_)`だとallが元の値  
`"Yota"`は`['Y','o','t','a']`と一緒で`x:_`は`[x,..]`の糖衣構文

In [16]:
firstLetter :: String -> String
firstLetter "" = "Empty String"
-- firstLetter all@(x:xs) = "The first letter of " ++ all ++ " is " ++ [x]
-- xs使わないからこっちのが良さそう
firstLetter all@(x:_) = "The first letter of " ++ all ++ " is " ++ [x]

firstLetter "Yota"

"The first letter of Yota is Y"

### ガード

ガード(`|`以下の条件式)は最低でも1つのスペースが必要

In [18]:
checkBmi :: Double -> String
checkBmi bmi
    | bmi <= 18.5 = "good"
    | bmi <= 25.0 = "so so"
    | bmi <= 30.0 = "fat"
    | otherwise = "whale"
    
checkBmi 17.5
checkBmi 27
checkBmi 34

"good"

"fat"

"whale"

複数の引数にも対応

In [22]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | weight / height ^ 2 <= 18.5 = "good"
    | weight / height ^ 2 <= 25.0 = "soso"
    | weight / height ^ 2 <= 30 = "fat"
    | otherwise = "whale"
    

checkBmi 66 1.72

"soso"

中置関数のパターン

In [21]:
myCompare :: (Ord a) => a -> a-> Ordering
a `myCompare` b
    | a == b = EQ
    | a <= b = LT
    | otherwise = GT
    
myCompare 3 3
myCompare 3 6
myCompare 3 2
    

EQ

LT

GT

### where

whereキーワードを使って計算の中間結果に名前をつける

In [54]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | bmi <= 18.5 = "good"
    | bmi <= 25.0 = "soso"
    | bmi <= 30 = "fat"
    | otherwise = "whale"
    where bmi = weight / height ^ 2
    
checkBmi 66 1.72

"soso"

なんかwhereのなかで複数の宣言するとエラーでる  
→原因はインデントの位置を揃えないといけなかった

In [14]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | bmi <= skinny = "good"
    | bmi <= 25.0 = "soso"
    | bmi <= 30 = "fat"
    | otherwise = "whale"
    where
        bmi = weight / height ^ 2
        skinny = 18.5
        normal = 25.0
        fat = 30
    
checkBmi 66 1.72

"soso"

whereのスコープ  
where節で定義した変数はその関数からしか見えない  
  
パターンマッチの関数でも同様で、下記だと違うパターンの関数ではniceは共有されないので使えない

In [1]:
greet :: String -> String
greet "Yota" = nice ++ " Yota"
greet "Hada" = nice ++ " Hada"
greet name = bad ++ " " ++ name
    where nice = "Nice"
                bad = "Bad"
                
greet "Yota"

: 

グローバルに持つ必要がある

In [2]:
nice :: String
nice = "Nice"
bad :: String
bad = "Bad"

greet "Y" = nice ++ " Y"
greet "H" = nice ++ " H"
greet name = bad ++ " " ++ name

greet "Y"
greet "other"

"Nice Y"

"Bad other"

パターンマッチとwhere  
where節の中でパターンマッチの宣言が可能  
イニシャルを返す関数

In [13]:
initials :: String -> String -> String
initials fn ln = [f] ++ ". " ++ [l] ++ "."
    where
        (f:_) = fn
        (l:_) = ln

initials "yota" "hada"

"y. h."

bmiのやつ

In [15]:
checkBmi :: Double -> Double -> String
checkBmi weight height
    | bmi <= s = "good"
    | bmi <= n = "soso"
    | bmi <= f = "fat"
    | otherwise = "whale"
    where
        bmi = weight / height ^ 2
        (s, n, f) = (18.5, 25.0, 30.0)
    
checkBmi 66 1.72

"soso"

whereブロックの中の関数  
体重と身長のペアのリストを受け取ってbmiを返す

In [17]:
chBmis :: [(Double, Double)] -> [Double]
chBmis xs = [bmi w h | (w, h) <- xs]
    where bmi weight height = weight / height ^ 2

chBmis [(66, 1.72), (75, 1.69)]

[22.30935640886966,26.25958474843318]

#### where節メモ

where節で定義した変数は関数内やガードでは使える  
ただスコープとして、パターンマッチにて別パターンの中では使えない  
where節で複数の変数がある場合はwhereだけ宣言して改行して変数のインデントを揃えた方がいいかも  

### let

where節は関数の終わりに定義  
定義したヘ変数はガードを含む関数全体から見える  
  
letはどこでも変数を定義できる  
ただガード間で共有はされない

円柱の表面積を高さと半径から求める  
`（円柱の表面積）= 2πrh(側面席) + 2πrの二乗`

let内で変数を定義しinで実行する内容をかく

In [20]:
cylinder :: Double -> Double -> Double
cylinder r h =
    let
        sideArea = 2 * pi * r * h
        topArea = pi * r ^ 2
    in sideArea + 2 * topArea

cylinder 3 3

113.09733552923255

letはwhereと違ってコード中のほとんどどんな場所でも使える

In [22]:
4 * (let a = 8 in a + 1) + 2

38

letはローカルスコープの関数を作るのに使える

In [24]:
[let square x = x * x in (square 5, square 7, square 11)]

[(25,49,121)]

セミコロンが使えるので複数の変数を一行で定義できる

In [25]:
let a = 1; b = 2; c =3 in a*b*c

6

let式 + パターンマッチでタプルを要素に分解して変数に格納できる

In [26]:
(let (a, b, c) = (1, 2, 3) in a + b + c) + 10

16

リスト内包表記  
下記のlet式はフィルタではなく、単なる変数宣言のみ

In [28]:
calcBmis :: [(Double, Double)] -> [Double]
calcBmis xs = [bmi | (w, h) <- xs, let bmi = w / h ^ 2]

calcBmis [(66, 1.72), (75, 1.69)]

[22.30935640886966,26.25958474843318]

フィルタもつかえる

In [29]:
calcBmis :: [(Double, Double)] -> [Double]
calcBmis xs = [bmi | (w, h) <- xs, let bmi = w / h ^ 2, bmi <= 23]

calcBmis [(66, 1.72), (75, 1.69)]

[22.30935640886966]

### case式

リストから先頭の要素を取り出すheadをcaseで書く

```
case expression of
    pattern -> result
    pattern -> result
    ...
```

In [31]:
head' :: [a] -> a
head' xs = case xs of
    [] -> error "error"
    (x:_) -> x

head' [1,2,3]

1

case式はどこでも使える  
デフォルトの指定の仕方

In [32]:
chList :: [a] -> String
chList ls = "The list is " ++ case ls of 
    [] -> "empty."
    [x] -> "a singleton list"
    --  ここがdefaultにあたる
    xs -> "a longer list"

chList []
chList [1]
chList [1,2,3]

"The list is empty."

"The list is a singleton list"

"The list is a longer list"

関数定義のパターンマッチでも同じことできるよ

In [34]:
chList :: [a] -> String
chList ls = "The list is " ++ what ls
    where
        what [] = "empty."
        what [x] = "a singleton list"
        what xs = "a longer list"
        
chList []
chList [1]
chList [1,2,3]

"The list is empty."

"The list is a singleton list"

"The list is a longer list"